In [2]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
# from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
import cv2
from sklearn.cluster import DBSCAN
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

pose_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_pose/session_pose_info'
os.makedirs(pose_analysis_session_data,exist_ok=True)

case_study_input_data = f'{SERVER_CACHE_DIR}/case_studies_input_raw/'
os.makedirs(case_study_input_data,exist_ok=True)


embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps_new'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

in_session_median_embeddings_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/in_session_median_embeddings'
os.makedirs(in_session_median_embeddings_data_dir, exist_ok=True)

in_session_cluster_embeddings_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/in_session_cluster_embeddings'
os.makedirs(in_session_cluster_embeddings_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_tracking/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)

/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
session_filter_list = [
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
]

In [4]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]        
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()

dict_keys(['05748A', '05410B', '17214C', '17214B', '05681A', '24352A', '24352C', '86375A'])

# Get ID level data across all sessions

In [18]:
sample_course = '05748A'
sample_session_id = 'classinsight-cmu_05748A_ghc_4101_201903071630-front'
session_tracking_cache_file = f"{track_analysis_session_data}/{sample_session_id}.pb"
session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{sample_session_id}.pb"
session_pose_file = f'{pose_analysis_session_data}/{sample_session_id}.pb'
session_frame_dir = f'{base_dir}/{sample_course}/{sample_session_id}'
session_video_file = f'/mnt/ci-nas-classes/classinsight/2019S/video_backup/{sample_session_id.split("-front")[0]}/{sample_session_id}.avi'
session_frame_dir

'/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face//05748A/classinsight-cmu_05748A_ghc_4101_201903071630-front'

In [19]:
old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
df_session_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{sample_session_id}.csv")
# Correct new ids with dict from eligible pairs
eligible_id_map_dict = {}
for id_pair in df_session_eligible_pairs.id_pair.values:
    (id1, id2) = eval(id_pair)
    print(id1, id2)
    if id2 in eligible_id_map_dict:
        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
    else:
        eligible_id_map_dict[id2] = id1

sprint(eligible_id_map_dict)

# correct created eligible map for once more
for key in sorted(list(eligible_id_map_dict.keys())):
    key_value = eligible_id_map_dict[key]
    if key_value in eligible_id_map_dict.keys():
        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
sprint(eligible_id_map_dict) 
# sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 
sprint("replacing ids in old to new id maps")
for old_id in old_to_new_id_map:
    if old_to_new_id_map[old_id] in eligible_id_map_dict:
        print(f"replacing {old_id}:{old_to_new_id_map[old_id]} -->{eligible_id_map_dict[old_to_new_id_map[old_id]]}")
        old_to_new_id_map[old_id] = eligible_id_map_dict[old_to_new_id_map[old_id]]
# sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 


16 25
19 25
26 31
Dict: eligible_id_map_dict
Key: Value


{19: 16, 25: 16, 31: 26}

Dict: eligible_id_map_dict
Key: Value


{19: 16, 25: 16, 31: 26}

"replacing ids in old to new id maps" : replacing ids in old to new id maps
replacing 987:19 -->16
replacing 1593:25 -->16
replacing 1917:31 -->26
replacing 1674:25 -->16


In [20]:
# map processed ids to tracking df
df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
total_idxs = df_tracking_new.index.max()
for old_id in old_to_new_id_map:
    new_id = old_to_new_id_map[old_id]
    if not new_id==10000:
        new_id_col = f'N{new_id}'
        if new_id_col not in df_tracking_new:
            df_tracking_new[new_id_col] = None
        df_tracking_new[new_id_col] =  df_tracking_new[new_id_col].where(~df_tracking_new[new_id_col].isnull(), df_tracking_new[old_id])
    df_tracking_new = df_tracking_new.drop(old_id, axis=1)




In [21]:
col_start_stop_idxs = []
for col in df_tracking_new.columns:
    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
df_id_start_stop

,id,min_idx,max_idx,total_idxs
0,0,6,66774,66768
1,1,9,66774,66765
2,2,30,7335,7305
3,3,30,66774,66744
4,4,30,66774,66744
5,5,288,45291,45003
6,6,318,66774,66456
7,7,360,66774,66414
8,8,366,18186,17820
9,9,420,66774,66354


In [22]:
INSTRUCTOR_ID_THRESHOLD = df_id_start_stop.total_idxs.max()*0.1
instructor_ids = df_id_start_stop[df_id_start_stop.total_idxs < INSTRUCTOR_ID_THRESHOLD]['id'].values
instructor_ids

array([12, 15, 18, 20, 21, 22, 24, 27, 30])

In [23]:
session_pose_info = pickle.load(open(f'{pose_analysis_session_data}/{sample_session_id}.pb','rb'))

In [46]:
# reorg session pose info
session_pose_info = {xr: 
                     {old_to_new_id_map[session_pose_info[xr]['people'][y_idx]['track_id']]:session_pose_info[xr]['people'][y_idx] 
                          for y_idx in range(len(session_pose_info[xr]['people'])) if 
                                (not (old_to_new_id_map[session_pose_info[xr]['people'][y_idx]['track_id']]==10000))}
                    for xr in session_pose_info}

KeyError: 'people'

In [47]:
session_pose_info[66774][23]['inference'], session_pose_info[66774][23].keys()

({'posture': {'armPose': 'armsCrossed', 'sitStand': 'sit'},
  'face': {'boundingBox': ((2708, 507), (2878, 677)), 'orientation': 'front'},
  'head': {},
  'trackingId': 231},
 dict_keys(['track_id', 'bbox', 'rvec', 'face_bb', 'gaze_2d', 'face_embedding', 'posenet_pose', 'body', 'inference']))

In [45]:
# arrange info as per tracking id across both sessions
gaze_info = {}
emb_info = {}
bbox_info = {}
face_info = {}
posture_info = {}
for frame_number in session_pose_info:
    for trackId in session_pose_info[frame_number]:
        if trackId not in gaze_info:
            gaze_info[trackId] = []
            emb_info[trackId]=[]
            bbox_info[trackId]= []
            face_info[trackId]= []
            posture_info[trackId]=[]
        # get  gaze info
        try:
            id_bbox = session_pose_info[frame_number][trackId]['bbox']
            bbox_info[trackId].append([frame_number]+list(id_bbox))

            id_face = session_pose_info[frame_number][trackId]['face_bb'][0]
            face_info[trackId].append([frame_number]+list(id_face))
            
            pitch, roll, yaw= session_pose_info[frame_number][trackId]['rvec'][0]
            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
            gaze_sx, gaze_sy, gaze_ex, gaze_ey = session_pose_info[frame_number][trackId]['gaze_2d'][0].flatten()
            gaze_info[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
            face_emb = session_pose_info[frame_number][trackId]['face_embedding'].tolist()
            emb_info[trackId].append([frame_number]+face_emb)

            armPose = 'other' if 'inference' not in session_pose_info[frame_number][trackId] else session_pose_info[frame_number][trackId]['inference']['posture']['armPose']
            sitStand = 'other' if 'inference' not in session_pose_info[frame_number][trackId] else session_pose_info[frame_number][trackId]['inference']['posture']['sitStand']
            posture_info[trackId].append([frame_number, armPose, sitStand])
        except:
            continue

for id in gaze_info:
    gaze_info[id] = pd.DataFrame(gaze_info[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
    emb_info[id] =pd.DataFrame(emb_info[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
    bbox_info[id] = pd.DataFrame(bbox_info[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')
    face_info[id] = pd.DataFrame(face_info[id], columns=['frame']+np.arange(15).tolist()).set_index('frame')
    posture_info[id] = pd.DataFrame(posture_info[id], columns=['frame','armPose','sitStand']).set_index('frame')

sprint({xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], posture_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info})


Dict: {xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], posture_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info}
Key: Value


{0: (21108, 21108, 21108, 21108, 22240),
 1: (22206, 22206, 22206, 22206, 22230),
 2: (1982, 1982, 1982, 1982, 2285),
 3: (20747, 20747, 20747, 20747, 22234),
 4: (22002, 22002, 22002, 22002, 22246),
 5: (14578, 14578, 14578, 14578, 14976),
 6: (21289, 21289, 21289, 21289, 22146),
 7: (20913, 20913, 20913, 20913, 22122),
 8: (5794, 5794, 5794, 5794, 5915),
 9: (21915, 21915, 21915, 21915, 22114),
 10: (14679, 14679, 14679, 14679, 15154),
 11: (20993, 20993, 20993, 20993, 21447),
 12: (54, 54, 54, 54, 259),
 13: (20746, 20746, 20746, 20746, 21582),
 14: (15594, 15594, 15594, 15594, 15685),
 15: (383, 383, 383, 383, 384),
 16: (15612, 15612, 15612, 15612, 15709),
 17: (15485, 15485, 15485, 15485, 15740),
 18: (307, 307, 307, 307, 307),
 20: (98, 98, 98, 98, 563),
 21: (221, 221, 221, 221, 480),
 22: (1286, 1286, 1286, 1286, 1420),
 23: (6880, 6880, 6880, 6880, 7162),
 24: (263, 263, 263, 263, 871),
 26: (5054, 5054, 5054, 5054, 5055),
 27: (21, 21, 21, 21, 300),
 28: (5065, 5065, 5065, 5065, 5196),
 29: (533, 533, 533, 533, 2312),
 30: (448, 448, 448, 448, 1735)}

In [51]:
for sid in posture_info:
    if sid not in instructor_ids:
        sprint(sid, posture_info[sid]['armPose'].value_counts(),posture_info[sid]['sitStand'].value_counts(),'\n') 

sid, posture_info[sid]['armPose'].value_counts(),posture_info[sid]['sitStand'].value_counts(),'\n' : 0 armsCrossed    11114
other           8675
handsOnFace     1193
handsRaised      104
error             22
Name: armPose, dtype: int64 sit      13999
error     6996
stand      113
Name: sitStand, dtype: int64 

sid, posture_info[sid]['armPose'].value_counts(),posture_info[sid]['sitStand'].value_counts(),'\n' : 1 other          12378
armsCrossed     4717
error           3040
handsOnFace     1856
handsRaised      215
Name: armPose, dtype: int64 error    21510
stand      417
sit        279
Name: sitStand, dtype: int64 

sid, posture_info[sid]['armPose'].value_counts(),posture_info[sid]['sitStand'].value_counts(),'\n' : 2 other          1332
armsCrossed     480
handsRaised     104
error            50
handsOnFace      16
Name: armPose, dtype: int64 sit      1440
error     492
stand      50
Name: sitStand, dtype: int64 

sid, posture_info[sid]['armPose'].value_counts(),posture_info[sid]['sitS

In [56]:
posture_info[0][posture_info[0]]

array([    6,     9,    12,    15,    18,    21,    30,    33,    36,
          39,    42,    45,    51,    84,   102,   108,   114,   117,
         120,   123,   129,   135,   138,   141,   144,   147,   150,
         153,   156,   159,   162,   165,   168,   171,   174,   177,
         180,   183,   186,   189,   192,   195,   198,   201,   204,
         207,   210,   213,   216,   219,   222,   225,   228,   231,
         234,   237,   240,   243,   246,   249,   252,   255,   390,
         393,   396,   399,   402,   432,   600,   777,   795,   798,
         804,   852,   855,  1029,  1041,  1152, 12954, 12957, 30183,
       30417, 30450, 30459, 30462, 30495, 30873, 42621, 42624, 42627,
       42630, 42633, 42636, 42639, 42642, 42645, 42648, 42687, 42690,
       42693, 42696, 42699, 42702, 46665, 46674, 46677, 46680, 46824,
       46830, 48942, 56118, 56127, 57468])

# Get case study information across all sessions


In [5]:
for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        case_study_input_cache_file = f"{case_study_input_data}/{session_id}.pb"
        try:
            if not os.path.exists(case_study_input_cache_file):
                printm(f"## Building case study session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
                session_pose_info = pickle.load(open(f'{pose_analysis_session_data}/{session_id}.pb','rb'))
                session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
                session_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{session_id}.pb","rb"))
                df_session_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{session_id}.csv")

                # Correct new ids with dict from eligible pairs
                eligible_id_map_dict = {}
                for id_pair in df_session_eligible_pairs.id_pair.values:
                    (id1, id2) = eval(id_pair)
                    print(id1, id2)
                    if id2 in eligible_id_map_dict:
                        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
                    else:
                        eligible_id_map_dict[id2] = id1
                
                sprint(eligible_id_map_dict)

                # correct created eligible map for once more
                for key in sorted(list(eligible_id_map_dict.keys())):
                    key_value = eligible_id_map_dict[key]
                    if key_value in eligible_id_map_dict.keys():
                        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
                sprint("Corrected", eligible_id_map_dict) 

                sprint("replacing ids in old to new id maps")
                for old_id in session_id_map:
                    if session_id_map[old_id] in eligible_id_map_dict:
                        print(f"replacing {old_id}:{session_id_map[old_id]} -->{eligible_id_map_dict[session_id_map[old_id]]}")
                        session_id_map[old_id] = eligible_id_map_dict[session_id_map[old_id]]

                printm("### map processed ids to tracking df")
                df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
                total_idxs = df_tracking_new.index.max()
                for old_id in session_id_map:
                    new_id = session_id_map[old_id]
                    if not new_id==10000:
                        new_id_col = f'N{new_id}'
                        if new_id_col not in df_tracking_new:
                            df_tracking_new[new_id_col] = None
                        df_tracking_new[new_id_col] =  df_tracking_new[new_id_col].where(~df_tracking_new[new_id_col].isnull(), df_tracking_new[old_id])
                    df_tracking_new = df_tracking_new.drop(old_id, axis=1)

                printm('### get id start stop dataframe')
                col_start_stop_idxs = []
                for col in df_tracking_new.columns:
                    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
                    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
                df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
                df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
                df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))

                printm('## Get instructor IDs')
                INSTRUCTOR_ID_THRESHOLD = df_id_start_stop.total_idxs.max()*0.1
                instructor_ids = df_id_start_stop[df_id_start_stop.total_idxs < INSTRUCTOR_ID_THRESHOLD]['id'].values
                sprint(instructor_ids)

                printm("### reorg session pose info")
                session_pose_info = {xr: 
                     {session_id_map[session_pose_info[xr]['people'][y_idx]['track_id']]:session_pose_info[xr]['people'][y_idx] 
                          for y_idx in range(len(session_pose_info[xr]['people'])) if 
                                (not (session_id_map[session_pose_info[xr]['people'][y_idx]['track_id']]==10000))}
                    for xr in session_pose_info}

                
                printm("### arrange info as per tracking id across both sessions")
                gaze_info = {}
                emb_info = {}
                bbox_info = {}
                face_info = {}
                posture_info = {}
                for frame_number in session_pose_info:
                    for trackId in session_pose_info[frame_number]:
                        if trackId not in gaze_info:
                            gaze_info[trackId] = []
                            emb_info[trackId]=[]
                            bbox_info[trackId]= []
                            face_info[trackId]= []
                            posture_info[trackId]=[]
                        # get  gaze info
                        try:
                            id_bbox = session_pose_info[frame_number][trackId]['bbox']
                            bbox_info[trackId].append([frame_number]+list(id_bbox))
                
                            id_face = session_pose_info[frame_number][trackId]['face_bb'][0]
                            face_info[trackId].append([frame_number]+list(id_face))
                            
                            pitch, roll, yaw= session_pose_info[frame_number][trackId]['rvec'][0]
                            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
                            gaze_sx, gaze_sy, gaze_ex, gaze_ey = session_pose_info[frame_number][trackId]['gaze_2d'][0].flatten()
                            gaze_info[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
                            face_emb = session_pose_info[frame_number][trackId]['face_embedding'].tolist()
                            emb_info[trackId].append([frame_number]+face_emb)
                
                            armPose = 'other' if 'inference' not in session_pose_info[frame_number][trackId] else session_pose_info[frame_number][trackId]['inference']['posture']['armPose']
                            sitStand = 'other' if 'inference' not in session_pose_info[frame_number][trackId] else session_pose_info[frame_number][trackId]['inference']['posture']['sitStand']
                            EdusenseTrackingId = -1
                            try:
                                EdusenseTrackingId = -1 if 'inference' not in session_pose_info[frame_number][trackId] else session_pose_info[frame_number][trackId]['inference']['trackingId']
                            except:
                                ...
                            posture_info[trackId].append([frame_number, armPose, sitStand, EdusenseTrackingId])
                        except:
                            continue
                
                for id in gaze_info:
                    gaze_info[id] = pd.DataFrame(gaze_info[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
                    emb_info[id] =pd.DataFrame(emb_info[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
                    bbox_info[id] = pd.DataFrame(bbox_info[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')
                    face_info[id] = pd.DataFrame(face_info[id], columns=['frame']+np.arange(15).tolist()).set_index('frame')
                    posture_info[id] = pd.DataFrame(posture_info[id], columns=['frame','armPose','sitStand','edusenseTrackingId']).set_index('frame')
                
                sprint({xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], posture_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info})
                
                session_case_study_input = (instructor_ids,gaze_info,emb_info,face_info,posture_info,bbox_info)                
                pickle.dump(session_case_study_input, open(case_study_input_cache_file,'wb'))
                printm(f"### Got cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                printm(f"## FILE EXISTS: cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get cross session input info for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            print(traceback.format_exc())
            

Session classinsight-cmu_05748A_ghc_4101_201902051630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902141630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201901291630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201901311630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902071630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902121630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902191630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902211630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902281630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903051630-front not in session filter list, skipping...


## Building case study session input info for session: 0-05748A, session:10-classinsight-cmu_05748A_ghc_4101_201903071630-front

16 25
19 25
26 31
Dict: eligible_id_map_dict
Key: Value


{19: 16, 25: 16, 31: 26}

"Corrected", eligible_id_map_dict : Corrected {25: 16, 19: 16, 31: 26}
"replacing ids in old to new id maps" : replacing ids in old to new id maps
replacing 987:19 -->16
replacing 1593:25 -->16
replacing 1917:31 -->26
replacing 1674:25 -->16


...

### arrange info as per tracking id across both sessions

Dict: {xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], posture_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info}
Key: Value


{0: (21108, 21108, 21108, 21108, 22240),
 1: (22206, 22206, 22206, 22206, 22230),
 2: (1982, 1982, 1982, 1982, 2285),
 3: (20747, 20747, 20747, 20747, 22234),
 4: (22002, 22002, 22002, 22002, 22246),
 5: (14578, 14578, 14578, 14578, 14976),
 6: (21289, 21289, 21289, 21289, 22146),
 7: (20913, 20913, 20913, 20913, 22122),
 8: (5794, 5794, 5794, 5794, 5915),
 9: (21915, 21915, 21915, 21915, 22114),
 10: (14679, 14679, 14679, 14679, 15154),
 11: (20993, 20993, 20993, 20993, 21447),
 12: (54, 54, 54, 54, 259),
 13: (20746, 20746, 20746, 20746, 21582),
 14: (15594, 15594, 15594, 15594, 15685),
 15: (383, 383, 383, 383, 384),
 16: (15612, 15612, 15612, 15612, 15709),
 17: (15485, 15485, 15485, 15485, 15740),
 18: (307, 307, 307, 307, 307),
 20: (98, 98, 98, 98, 563),
 21: (221, 221, 221, 221, 480),
 22: (1286, 1286, 1286, 1286, 1420),
 23: (6880, 6880, 6880, 6880, 7162),
 24: (263, 263, 263, 263, 871),
 26: (5054, 5054, 5054, 5054, 5055),
 27: (21, 21, 21, 21, 300),
 28: (5065, 5065, 5065, 5065, 5196),
 29: (533, 533, 533, 533, 2312),
 30: (448, 448, 448, 448, 1735)}

### Got cross session input info for session: 0-05748A, session:10-classinsight-cmu_05748A_ghc_4101_201903071630-front

Session classinsight-cmu_05748A_ghc_4101_201903121630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903141630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903191630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903211630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903261630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201903281630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201904021630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201904041630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201904091630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201904111630-front not in session filter list, skipping...


In [6]:
test_session_id = 'classinsight-cmu_05748A_ghc_4101_201904021630-front'
(instructor_ids, _,_,_,_,_) = pickle.load(open(f"{case_study_input_data}/{test_session_id}.pb",'rb'))

In [7]:
instructor_ids

array([15, 18, 19, 20, 21, 23, 27, 29, 32, 33, 34, 35])